## MODNet


In [ ]:
import os

# clone the repository
if not os.path.exists('MODNet'):
  !git clone https://github.com/ZHKKKe/MODNet

%cd MODNet/

# dowload the pre-trained ckpt for image matting
pretrained_ckpt = 'pretrained/modnet_photographic_portrait_matting.ckpt'
if not os.path.exists(pretrained_ckpt):
  !gdown --id 1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz \
          -O pretrained/modnet_photographic_portrait_matting.ckpt


## Inference

<p align="justify">Run the following command for alpha matte prediction:</p>

In [2]:

%cd MODNet/
# input image path 지정
# output 폴더 만들어두고 지정
!python -m demo.image_matting.colab.inference \
        --input-path ../../data/images \
        --output-path ../../data/mask \
        --ckpt-path ./pretrained/modnet_photographic_portrait_matting.ckpt


[Errno 2] No such file or directory: 'MODNet/'
/home/cvmlserver/Seohyeon/smplpix/data_generation/MODNet
Process image: 00175.png
Process image: 00856.png
Process image: 01031.png
Process image: 00974.png
Process image: 00917.png
Process image: 00270.png
Process image: 00214.png
Process image: 00660.png
Process image: 00878.png
Process image: 00121.png
Process image: 00560.png
Process image: 00069.png
Process image: 00363.png
Process image: 00644.png
Process image: 00048.png
Process image: 00784.png
Process image: 00836.png
Process image: 00092.png
Process image: 00013.png
Process image: 00019.png
Process image: 00367.png
Process image: 00247.png
Process image: 00949.png
Process image: 00746.png
Process image: 00532.png
Process image: 00677.png
Process image: 00278.png
Process image: 00807.png
Process image: 00157.png
Process image: 00849.png
Process image: 00985.png
Process image: 00115.png
Process image: 00798.png
Process image: 00531.png
Process image: 00972.png
Process image: 00059.

In [3]:
import numpy as np
from PIL import Image
import cv2  
# 아래 경로 세 개 바꿔주면 됨
# extract backgrounds from input using mask
def combined_display(image, matte, image_name):
    w, h, c = image.shape
    rw, rh = 800, int(h * 800 / (3 * w))
    
    (thresh, matte) = cv2.threshold(matte, w, h, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    thresh = 255 - thresh
    matte = cv2.threshold(matte, thresh, 255, cv2.THRESH_BINARY)[1]
    foreground = cv2.bitwise_and(image, image, mask=matte) 
    white_back = np.ones_like(image, np.uint8) * 255
    background = cv2.bitwise_and(white_back, white_back, mask=255-matte)
    result = cv2.add(foreground, background)
    # 최종 결과 폴더 만들어두고 지정
    cv2.imwrite('../../data/final/'+image_name, result)

# input_folder = image(원본)
# output_folder = mask(마스크)
input_folder = '../../data/images/'
output_folder = '../../data/mask/'
# visualize all images
image_names = os.listdir(input_folder)

for image_name in image_names:
    matte_name = image_name.split('.')[0] + '.png'
    image = cv2.imread(os.path.join(input_folder, image_name))
    matte = cv2.imread(os.path.join(output_folder, matte_name),cv2.IMREAD_GRAYSCALE)
    #  matte = cv2.cvtColor(matte, cv2.COLOR_GRAY2BGR)

    
    combined_display(image, matte, matte_name)
